# data

In [1]:
from os import listdir
from os.path import isfile, join


import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


In [2]:
ham = [f for f in listdir("data/datasets-master/email/plaintext/corpus1/ham") \
             if isfile(join("data\datasets-master\email\plaintext\corpus1\ham", f))]
spam = [f for f in listdir("data/datasets-master/email/plaintext/corpus1/spam") \
             if isfile(join("data\datasets-master\email\plaintext\corpus1\spam", f))]
for index in range(len(ham)):
    ham[index] = "data/datasets-master/email/plaintext/corpus1/ham/"+ ham[index]
for index in range(len(spam)):
    spam[index] = "data/datasets-master/email/plaintext/corpus1/spam/"+ spam[index]

In [3]:
from sklearn.model_selection import train_test_split
Training_Ham, Testing_Ham = train_test_split(ham, test_size = 0.1, random_state = 1)
Training_Spam, Testing_Spam = train_test_split(spam, test_size = 0.1, random_state = 1)

In [4]:
print(len(Training_Ham), len(Testing_Ham), len(Training_Spam), len(Testing_Spam))

3304 368 1350 150


In [5]:
import nltk
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
s=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
import re
import codecs
frequency_list = [1, 2, 3, 5, 10, 20, 30, 50, 100, 200, 300, 500, 1000]
# frequency_list = [500]

dictionary = dict()
Training = Training_Ham + Training_Spam
Testing = Testing_Ham + Testing_Spam


In [7]:

from sklearn.ensemble import RandomForestClassifier
f = open("ret/RF_data2.txt", "a")
for frequency in frequency_list:
    print('frequency is ' + str(frequency))
    f.write('\nfrequency is ' + str(frequency) + '\n')
    dictionary = dict()
    
    for file_names in Training:
        file = codecs.open(file_names, 'r', encoding='utf-8', errors='ignore')
        content = file.read()
        find_sub = re.findall(r"subject : .*\n", content)
        if len(find_sub) == 0:
            find_sub = re.findall(r"Subject: .*\n", content)
            body = content.split(find_sub[-1])[1].lower()
        else:
            body = content.split(find_sub[-1])[1].lower()
        #word = re.findall(r"[\w']+", body) # 93783 ## 21525
        word = re.findall(r"[a-zA-Z']+", body) # 123656 ## 21224
#         word = body.split(' ')
#         lemma = WordNetLemmatizer()
#         word = [lemma.lemmatize(w, pos = "v") for w in word]
#         word = [lemma.lemmatize(w, pos = "n") for w in word]
        for i in word:
            dictionary[i] = dictionary.get(i, 0) + 1
#     print(len(dictionary))
    dictionary = {i:dictionary[i] for i in dictionary if dictionary[i]>frequency}
#     print(len(dictionary))
    #print(dictionary)
    list_dictionary = list(dictionary)
    list_dictionary = list(filter(lambda w: not w in s,list_dictionary))
    list_dictionary
#     print(len(list_dictionary))
    
    # Remove common word in dictionary
    for s__ in s:
        if s__ in dictionary:
            del dictionary[s__]

    print('dict length: ' + str(len(dictionary)))
    f.write('dict length: ' + str(len(dictionary)))
    count = 0
    for i in dictionary:
        dictionary[i] = count
        count += 1

    label_train = [0] * len(Training_Ham) + [1] * len(Training_Spam)
    label_test = [0] * len(Testing_Ham) + [1] * len(Testing_Spam)
        
# ****************** train *****************
    train_ndarray = np.zeros((len(Training), len(dictionary)), dtype = int)

    count = 0
    for Ham_mail in Training_Ham:
        file = codecs.open(Ham_mail, 'r', encoding='utf-8', errors='ignore')
        content = file.read()
        find_sub = re.findall(r"subject : .*\n", content)
        if len(find_sub) == 0:
            find_sub = re.findall(r"Subject: .*\n", content)
            body = content.split(find_sub[-1])[1].lower()
        else:
            body = content.split(find_sub[-1])[1].lower()
        word = re.findall(r"[a-zA-Z']+", body)
#         word = body.split(' ')
#         lemma = WordNetLemmatizer()
#         word = [lemma.lemmatize(w, pos = "v") for w in word]
#         word = [lemma.lemmatize(w, pos = "n") for w in word]
        for w in word:
            if w in dictionary:
                train_ndarray[count][dictionary[w]] = 1
        count += 1
        
    for Spam_mail in Training_Spam:
        file = codecs.open(Spam_mail, 'r', encoding='utf-8', errors='ignore')
        content = file.read()
        find_sub = re.findall(r"subject : .*\n", content)
        if len(find_sub) == 0:
            find_sub = re.findall(r"Subject: .*\n", content)
            body = content.split(find_sub[-1])[1].lower()
        else:
            body = content.split(find_sub[-1])[1].lower()
        word = re.findall(r"[a-zA-Z']+", body)
#         word = body.split(' ')
#         lemma = WordNetLemmatizer()
#         word = [lemma.lemmatize(w, pos = "v") for w in word]
#         word = [lemma.lemmatize(w, pos = "n") for w in word]
        for w in word:
            if w in dictionary:
                train_ndarray[count][dictionary[w]] = 1
        count += 1
    train_dataframe = pd.DataFrame(train_ndarray)
    
# ****************** test ******************
    
    test_ndarray = np.zeros((len(Testing), len(dictionary)), dtype = int)
    count = 0
    for Ham_mail in Testing_Ham:
        file = codecs.open(Ham_mail, 'r', encoding='utf-8', errors='ignore')
        content = file.read()
        find_sub = re.findall(r"subject : .*\n", content)
        if len(find_sub) == 0:
            find_sub = re.findall(r"Subject: .*\n", content)
            body = content.split(find_sub[-1])[1].lower()
        else:
            body = content.split(find_sub[-1])[1].lower()
        word = re.findall(r"[a-zA-Z']+", body)
#         word = body.split(' ')
#         lemma = WordNetLemmatizer()
#         word = [lemma.lemmatize(w, pos = "v") for w in word]
#         word = [lemma.lemmatize(w, pos = "n") for w in word]
        for w in word:
            if w in dictionary:
                test_ndarray[count][dictionary[w]] = 1
        count += 1
          
    for Spam_mail in Testing_Spam:
        file = codecs.open(Spam_mail, 'r', encoding='utf-8', errors='ignore')
        content = file.read()
        find_sub = re.findall(r"subject : .*\n", content)
        if len(find_sub) == 0:
            find_sub = re.findall(r"Subject: .*\n", content)
            body = content.split(find_sub[-1])[1].lower()
        else:
            body = content.split(find_sub[-1])[1].lower()
        word = re.findall(r"[a-zA-Z']+", body)
#         word = body.split(' ')
#         lemma = WordNetLemmatizer()
#         word = [lemma.lemmatize(w, pos = "v") for w in word]
#         word = [lemma.lemmatize(w, pos = "n") for w in word]
        for w in word:
            if w in dictionary:
                test_ndarray[count][dictionary[w]] = 1
        count += 1
    test_dataframe = pd.DataFrame(test_ndarray)
    
# **************************************************************

    RF = RandomForestClassifier()
    RF.fit(train_dataframe, label_train)

    result = RF.predict(test_dataframe)

    
    #-------------- from block -----------------
    
#     Spam_from = dict()
#     count = 0
#     for Spam_mail in Training_Spam:
#         file = codecs.open(Spam_mail, 'r', encoding='utf-8', errors='ignore')
#         word = re.findall(r"From.*\n", file.read())[0]
#         word_2 = re.findall(r"[\w*\.?]*@[\w*\.?]*", word)
#         if not word_2 :
#             continue
#         Spam_from[word_2[0]] = Spam_from.get(i, 0)    
    
#     index = 0
#     cheat_result = result.copy()
#     for mail in Testing:
#         file = codecs.open(mail, 'r', encoding='utf-8', errors='ignore')
#         word = re.findall(r"From.*\n", file.read())[0]
#         word_2 = re.findall(r"[\w*\.?]*@[\w*\.?]*", word)
#         if word_2[0] in Spam_from:
#             cheat_result[index] = 1
#         index += 1
    
#     eva_fun_2(label_test, cheat_result, 'NN')
    
    #--------------- end ---------------------
    
       
    tn, fp, fn, tp = confusion_matrix(label_test, result).ravel()

    Accuracy = (tp+tn)/(tn+ fp+ fn+ tp)
    spam_Precision = (tp)/(fp+ tp)
    spam_Recall = (tp)/(fn+ tp)
    
    ham_Precision = (tn)/(fn + tn)
    ham_Recall = (tn)/(fp + tn)
    
    print('Spam\nRecall\t\tPrecision\tAccuracy')
    f.write('\n\nSpam\nRecall\t\tPrecision\tAccuracy\n')
    print(str(round(spam_Recall*100, 2))+'\t\t'+ str(round(spam_Precision*100, 2))+'\t\t'+str(round(Accuracy*100, 2)))
    f.write(str(round(spam_Recall*100, 2))+'\t\t'+ str(round(spam_Precision*100, 2))+'\t\t'+str(round(Accuracy*100, 2)))
    
    print('\nHam\nRecall\t\tPrecision\tAccuracy')
    f.write('\n\nHam\nRecall\t\tPrecision\tAccuracy\n')
    print(str(round(ham_Recall*100, 2))+'\t\t'+ str(round(ham_Precision*100, 2))+'\t\t'+str(round(Accuracy*100, 2)))
    f.write(str(round(ham_Recall*100, 2))+'\t\t'+ str(round(ham_Precision*100, 2))+'\t\t'+str(round(Accuracy*100, 2))+ '\n\n')
    
    print(tn, fp, fn, tp)
    f.write('tn ' + str(tn) + ' fp ' + str(fp) + ' fn ' + str(fn) + ' tp ' + str(tp))
    print('\n------------------------------------\n')
    f.write('\n------------------------------------\n')
f.close()

frequency is 1
dict length: 18010
Spam
Recall		Precision	Accuracy
95.33		92.86		96.53

Ham
Recall		Precision	Accuracy
97.01		98.08		96.53
357 11 7 143

------------------------------------

frequency is 2
dict length: 11564
Spam
Recall		Precision	Accuracy
96.67		92.36		96.72

Ham
Recall		Precision	Accuracy
96.74		98.61		96.72
356 12 5 145

------------------------------------

frequency is 3
dict length: 8633
Spam
Recall		Precision	Accuracy
95.33		92.86		96.53

Ham
Recall		Precision	Accuracy
97.01		98.08		96.53
357 11 7 143

------------------------------------

frequency is 5
dict length: 6130
Spam
Recall		Precision	Accuracy
95.33		92.86		96.53

Ham
Recall		Precision	Accuracy
97.01		98.08		96.53
357 11 7 143

------------------------------------

frequency is 10
dict length: 3944
Spam
Recall		Precision	Accuracy
95.33		92.26		96.33

Ham
Recall		Precision	Accuracy
96.74		98.07		96.33
356 12 7 143

------------------------------------

frequency is 20
dict length: 2395
Spam
Recall		Preci

In [8]:
tn, fp, fn, tp

(344, 24, 66, 84)

In [9]:
# count_list = []
# for j in range(len(dictionary)):
#     count = 0
#     for i in range(len(label_train)):
#         if (train_dataframe[j][i] == 1) & (label_train[i] == 1) :
#             count += 1
#     count_list.append(count)
#     if (count > 100):
#         print (j)

In [10]:
# arr = np.ones([(len(label_test))], dtype = bool)
# arr

In [11]:
# result = SVM.predict(test_dataframe)
# l = count_list.copy()
# arr = np.ones([(len(label_test))], dtype = bool)
# for i in range(3):
#     b = np.array(l)
#     Max = np.argmax(b)
#     l[Max] = -1
#     for i in range(len(label_test)):
#         if (test_dataframe[Max][i] != 1):
#             arr[i] = False
# for i in range(len(label_test)):
#     if arr[i]:
#         result[i] = 1

In [12]:
# tn, fp, fn, tp = confusion_matrix(label_test, result).ravel()

# Accuracy = (tp+tn)/(tn + fp + fn + tp)
# Precision = (tp)/(fp + tp)
# Recall = (tp)/(fn + tp)
# print('Recall\t\tPrecision\tAccuracy')
# # f.write('\nRecall\t\tPrecision\tAccuracy\n')
# print(str(round(Recall*100, 2))+'\t\t'+ str(round(Precision*100, 2))+'\t\t'+str(round(Accuracy*100, 2)))
# # f.write(str(round(Recall*100, 2))+'\t\t'+ str(round(Precision*100, 2))+'\t\t'+str(round(Accuracy*100, 2)))
# print('\n------------------------------------\n')
# # f.write('\n------------------------------------\n')

In [13]:
# pd.Series(arr).value_counts()